In [2]:
# 导入必要的库
import numpy as np
import pandas as pd
# 文件路径
TRAIN_PATH = './data/round1_ijcai_18_train_20180301.txt'
TEST_PATH = './data/round1_ijcai_18_test_a_20180301.txt'
# 读取数据
train_df = pd.read_csv(TRAIN_PATH, sep=r'\s+')
test_df = pd.read_csv(TEST_PATH, sep=r'\s+')
# 记录原始训练集长度，以便后续分割
train_len = len(train_df)
# 合并训练集和测试集
data_df = pd.concat([train_df.drop('is_trade', axis=1), test_df], ignore_index=True)
# 添加原始索引列，便于后续还原顺序
data_df['original_index'] = data_df.index

print(f'合并后的数据总行数: {len(data_df)}')

print('开始提取时间特征...')
# 1. 基础时间特征提取（使用Datetime对象）
data_df['context_timestamp'] = pd.to_datetime(data_df['context_timestamp'], unit='s')
data_df['context_hour'] = data_df['context_timestamp'].dt.hour
data_df['context_weekday'] = data_df['context_timestamp'].dt.dayofweek
data_df['context_month'] = data_df['context_timestamp'].dt.month
# 2. 计算深度时序特征
data_df = data_df.sort_values(by=['user_id', 'context_timestamp']).reset_index(drop=True)
# 计算用户和店铺的时间间隔特征（Last Interaction Delta）
for col in ['user_id', 'shop_id']:
    # 获取前一条记录的时间戳
    data_df[f'{col}_prev_time'] = data_df.groupby(col)['context_timestamp'].shift(1)
    # 计算时间间隔（Timedelta 对象）
    data_df[f'{col}_time_gap'] = data_df['context_timestamp'] - data_df[f'{col}_prev_time']
    # 删除临时列
    data_df.drop(f'{col}_prev_time', axis=1, inplace=True)
# 3. 将时间特征数值化（转换为秒，float）
# Timedelta 列转换为秒
data_df['user_delta_time'] = data_df.groupby('user_id')['context_timestamp'].diff().dt.total_seconds().fillna(-1)
data_df['user_id_time_gap'] = data_df['user_id_time_gap'].dt.total_seconds().fillna(-1)
data_df['shop_id_time_gap'] = data_df['shop_id_time_gap'].dt.total_seconds().fillna(-1)
# 将原始时间戳转换为数值（用于模型训练）
data_df['context_timestamp'] = data_df['context_timestamp'].astype(np.int64) // 10**9
# 还原数据顺序
data_df = data_df.sort_values(by='original_index').reset_index(drop=True)
data_df.drop('original_index', axis=1, inplace=True)

print('时序特征提取与数值化完成。')

合并后的数据总行数: 496509
开始提取时间特征...
时序特征提取与数值化完成。


In [3]:
# --- 序列特征：item_category_list ---
print('开始解析 item_category_list...')
# 提取主类目（第一个ID），如果列表为空则返回 NaN 或一个占位符
def get_first_category(cat_list):
    try:
        return cat_list.split(';')[0]
    except:
        return np.nan
# 提取次级类目（第二个ID）
def get_second_category(cat_list):
    try:
        # 如果列表中只有1个元素，split(';')[1]会出错，需try-except处理
        parts = cat_list.split(';')
        if len(parts) > 1:
            return parts[1]
        return np.nan
    except:
        return np.nan
# 提取类目深度（ID数量）
data_df['item_category_depth'] = data_df['item_category_list'].apply(
    lambda x: len(x.split(';')) if pd.notna(x) else 0
)
# 应用函数提取主/次类目
data_df['item_category_1'] = data_df['item_category_list'].apply(get_first_category)
data_df['item_category_2'] = data_df['item_category_list'].apply(get_second_category)
# 删除原始列
data_df.drop('item_category_list', axis=1, inplace=True)

print('item_category_list 解析完成。')
    

开始解析 item_category_list...
item_category_list 解析完成。


In [4]:
# --- 序列特征：item_property_list ---
print('开始解析 item_property_list...')
# 提取属性数量
data_df['item_property_count'] = data_df['item_property_list'].apply(
    lambda x: len(x.split(';')) if pd.notna(x) else 0
)
# 可选：提取第一个属性ID作为新特征
def get_first_property(prop_list):
    try:
        return prop_list.split(';')[0]
    except:
        return np.nan

data_df['item_property_1'] = data_df['item_property_list'].apply(get_first_property)
# 删除原始列
data_df.drop('item_property_list', axis=1, inplace=True)

print('item_property_list 解析完成。')

开始解析 item_property_list...
item_property_list 解析完成。


In [5]:
# --- 计数特征 (Count Features) ---
print("开始构建计数特征...")
# 需要统计的ID字段列表
id_features = [
    'user_id', 'item_id', 'shop_id', 
    'item_category_1', 'item_brand_id', 'item_city_id'
]
for col in id_features:
    # 构建新列名，例如：user_id_count
    new_col = f'{col}_count'
    # 使用 value_counts() 统计频次，并 map 回原 DataFrame
    count_map = data_df[col].value_counts().to_dict()
    data_df[new_col] = data_df[col].map(count_map)
    # 打印部分结果以供检查
    print(f"特征 {new_col} 构建完成。")

print("计数特征构建完成。")

开始构建计数特征...
特征 user_id_count 构建完成。
特征 item_id_count 构建完成。
特征 shop_id_count 构建完成。
特征 item_category_1_count 构建完成。
特征 item_brand_id_count 构建完成。
特征 item_city_id_count 构建完成。
计数特征构建完成。


In [6]:
# --- 交叉计数特征 (Cross Count Features) ---
print("开始构建交叉计数特征...")
# 需要统计交叉频次的特征对列表
cross_features = [
    ('user_id', 'shop_id'),
    ('user_id', 'item_category_1'),
    ('user_id', 'item_brand_id'),
    ('item_id', 'context_page_id')
]

for col1, col2 in cross_features:
    new_col = f'{col1}__{col2}_count'
    # 使用 groupby 进行交叉计数
    gp = data_df.groupby([col1, col2]).size()
    gp = gp.reset_index().rename(columns={0: new_col})
    # 合并回原 DataFrame
    data_df = data_df.merge(gp, on=[col1, col2], how='left')

    print(f"特征 {new_col} 构建完成。")

print("交叉计数特征构建完成。")

开始构建交叉计数特征...
特征 user_id__shop_id_count 构建完成。
特征 user_id__item_category_1_count 构建完成。
特征 user_id__item_brand_id_count 构建完成。
特征 item_id__context_page_id_count 构建完成。
交叉计数特征构建完成。


In [7]:
# --- 拆分回训练集和测试集 ---
train_processed = data_df.iloc[:train_len].copy()
test_processed = data_df.iloc[train_len:].copy()
# 将训练集的 is_trade 列合并回 train_processed
train_processed['is_trade'] = train_df['is_trade']
# --- 转化率特征 (Converrsion Rate Features) ---
print('开始构建转化率特征...')
# 需要计算转化率的特征列表
cr_features = ['user_id', 'shop_id', 'item_category_1']

for col in cr_features:
    new_col = f'{col}_CTR' # CTR 表示 Click-Through Rate，这里是交易率
    # 1. 在训练集上进行 Groupby 聚合
    #  agg(['mean']) 计算 is_trade 的均值即为转化率
    gp = train_processed.groupby(col)['is_trade'].agg(['mean']).reset_index()
    gp.rename(columns={'mean': new_col}, inplace=True)
    # 2. 将转化率特征合并回训练集和测试集
    train_processed = pd.merge(train_processed, gp, on=col, how='left')
    test_processed = pd.merge(test_processed, gp, on=col, how='left')

    print(f"特征 {new_col} 构建完成。")

print('转化率特征构建完成。')

开始构建转化率特征...
特征 user_id_CTR 构建完成。
特征 shop_id_CTR 构建完成。
特征 item_category_1_CTR 构建完成。
转化率特征构建完成。


In [8]:
# 1. 识别并删除不需要的列
# ID特征通常不需要直接输入树模型，但它们可以作为类别特征
# 先删除原始的、高基数的ID列，仅保留作为统计特征的ID列
DROP_COLS = [
    'instance_id', 'user_id', 'item_id', 'shop_id', 'context_id',
    'item_category_list', 'item_property_list', 'predict_category_property'
    # 之前已经删除了 item_category_list 和 item_property_list，这里确保其他高基数ID也处理
]
# 找到当前DataFrame中存在的ID列进行删除（防止重复删除报错）
cols_to_drop = [col for col in DROP_COLS if col in train_processed.columns]
x_train = train_processed.drop(cols_to_drop + ['is_trade'], axis=1)
y_train = train_processed['is_trade']
x_test = test_processed.drop(cols_to_drop, axis=1)

from sklearn.preprocessing import LabelEncoder
# --- 识别需要编码的特征类别 ---
CATEGORY_FEATURES = [
    'item_category_1', 
    'item_category_2', 
    'item_property_1',
    # 还可以加入其他高基数的ID特征，如 item_brand_id, item_city_id, etc.
    'item_brand_id', 'item_city_id' 
]

print("开始对高基数ID特征进行 Label Encoding...")
# 循环对需要编码的特征进行处理
for col in CATEGORY_FEATURES:
    # 确保列存在，并将其类型转换为字符串（以便于LablelEncoder处理）
    if col in x_train.columns:
        x_train[col] = x_train[col].astype(str)
        x_test[col] = x_test[col].astype(str)
        # 将训练集和测试集的当前列合并，确保编码器能看到所有可能的ID值
        full_data = pd.concat([x_train[col], x_test[col]], ignore_index=True)

        le = LabelEncoder()
        # fit on full data
        le.fit(full_data)
        # transform back
        x_train[col] = le.transform(x_train[col])
        x_test[col] = le.transform(x_test[col])
        # 将数据类型强制转换为整数
        x_train[col] = x_train[col].astype(int)
        x_test[col] = x_test[col].astype(int)

        print(f"特征 {col} 编码完成。")

print("所有类别特征编码完成。")
# 2. 处理缺失值
# 树模型对NaN值不敏感，但为了保险起见，填充一个特殊值
# 对于类别特征，用一个特殊的负值或‘NaN’字符串填充；对于数值特征，用-999等特殊值填充
x_train = x_train.fillna(-999)
x_test = x_test.fillna(-999)

x_test = x_test[x_train.columns]

print(f"训练集特征维度: {x_train.shape}")
print(f"测试集特征维度: {x_test.shape}")

开始对高基数ID特征进行 Label Encoding...
特征 item_category_1 编码完成。
特征 item_category_2 编码完成。
特征 item_property_1 编码完成。
特征 item_brand_id 编码完成。
特征 item_city_id 编码完成。
所有类别特征编码完成。
训练集特征维度: (478138, 42)
测试集特征维度: (18371, 42)


In [9]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
# 定义模型评估指标：AUC 和 LogLoss
# LightGBM 参数设置（基于经验）
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc', # 仍然使用 AUC 作为 early_stopping 的监控指标，更稳定
    'learning_rate': 0.03, # ⚠️ 降低学习率（原 0.05）
    'num_leaves': 31,
    'max_depth': -1,
    'seed': 42,
    'n_estimators': 2000, # ⚠️ 增加迭代次数（原 1000）
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'reg_alpha': 0.5,  # ⚠️ 增加 L1 正则化（原 0.1）
    'reg_lambda': 0.5, # ⚠️ 增加 L2 正则化（原 0.1）
    'n_jobs': -1,
    'verbose': -1
}
# --- 交叉验证训练（K-Fold Cross-Validation）---
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)

oof_preds = np.zeros(x_train.shape[0]) # Out-of-Fold 预测，用于模型融合
sub_preds = np.zeros(x_test.shape[0])  # 测试集预测

print("开始交叉验证训练...")

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(x_train, y_train)):
    x_train_fold, y_train_fold = x_train.iloc[train_idx], y_train.iloc[train_idx]
    x_valid_fold, y_valid_fold = x_train.iloc[valid_idx], y_train.iloc[valid_idx]
    # 初始化并训练模型
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(x_train_fold, y_train_fold,
              eval_set=[(x_valid_fold, y_valid_fold)],
              callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
    # 在验证集上进行预测
    oof_preds[valid_idx] = model.predict_proba(x_valid_fold)[:, 1]
    sub_preds += model.predict_proba(x_test)[:, 1] / folds.n_splits
    valid_preds = model.predict_proba(x_valid_fold)[:, 1]
    oof_preds[valid_idx] = valid_preds
    # 计算并打印当前折的评估指标
    valid_logloss = log_loss(y_valid_fold, valid_preds)
    valid_auc = roc_auc_score(y_valid_fold, valid_preds)

    print(f'Fold {n_fold + 1} - LogLoss: {valid_logloss:.6f}, AUC: {valid_auc:.6f}')
# --- 结果评估 --
overall_logloss = log_loss(y_train, oof_preds) # 计算整体 LogLoss
overall_auc = roc_auc_score(y_train, oof_preds)
print(f'Overall - LogLoss: {overall_logloss:.6f}, AUC: {overall_auc:.6f}')

开始交叉验证训练...
Fold 1 - LogLoss: 0.028820, AUC: 0.993530
Fold 2 - LogLoss: 0.024109, AUC: 0.994388
Fold 3 - LogLoss: 0.024649, AUC: 0.993860
Fold 4 - LogLoss: 0.025440, AUC: 0.993496
Fold 5 - LogLoss: 0.024550, AUC: 0.993784
Overall - LogLoss: 0.025514, AUC: 0.993803


In [10]:
# 提取特征名和重要性得分
feature_importances = pd.DataFrame(
    {'feature': x_train.columns, 'importance': model.feature_importances_}
).sort_values(by='importance', ascending=False)
# 打印前 20 个重要特征
print("前 20 个重要特征:")
print(feature_importances.head(20))

前 20 个重要特征:
                           feature  importance
40                     shop_id_CTR         472
22                shop_id_time_gap         449
21                user_id_time_gap         414
39                     user_id_CTR         403
10               context_timestamp         218
29                   user_id_count         175
35          user_id__shop_id_count         172
16             shop_score_delivery         150
18                    context_hour         148
38  item_id__context_page_id_count         145
0                    item_brand_id         143
30                   item_id_count         140
31                   shop_id_count         140
17          shop_score_description         127
27             item_property_count         122
15              shop_score_service         109
9                  user_star_level          95
13       shop_review_positive_rate          94
23                 user_delta_time          79
37    user_id__item_brand_id_count          78


In [11]:
# --- 生成提交文件 ---
submission = pd.DataFrame({
    'instance_id': test_df['instance_id'],
    'is_trade': sub_preds
})

SUBMISSION_PATH = './submission.csv'
submission.to_csv(SUBMISSION_PATH, index=False, float_format='%.16f') # 强制保留小数点后 10 位，确保精度
print(f"提交文件已保存至 {SUBMISSION_PATH}")

提交文件已保存至 ./submission.csv
